# Implementation: Trust and Tuning

**Goal**: Explain the unexplainable.

In [ ]:
import shap
import optuna
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np

# Mock Data and Model
X = np.random.rand(100, 4)
y = np.random.randint(10, 200, 100)
model = xgb.XGBRegressor().fit(X, y)

# 1. SHAP Explanation
explainer = shap.Explainer(model)
shap_values = explainer(X)

print("SHAP Values calculated. In a notebook, use shap.plots.beeswarm(shap_values) to visualize.")
print(f"Base Value (Average Prediction): {explainer.expected_value:.2f}")
print(f"Impact of Feature 0 on Sample 0: {shap_values[0].values[0]:.2f} days")

# 2. Optuna Optimization
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 200)
    }
    
    # Train with mocked CV
    model = xgb.XGBRegressor(**param)
    model.fit(X, y)
    preds = model.predict(X)
    return np.sqrt(mean_squared_error(y, preds))

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

print(f"Best RMSE: {study.best_value:.2f}")
print(f"Best Params: {study.best_params}")